# Module 4, Week 2 Homework

Non-linear model fitting

**Before class reading: Chapter C of USGS HayWired Report **

**In-class we:**
- Load peak ground acceleration observations from two notable M6 quakes in California
- Fit a ground motion prediction equation (GMPE) using `polyfit`
- Vary our assumed mean event depth to find better fitting model
- Fit a GMPE after weighing the data by the distance distribution using `linalg.solve`

**Our goals for this assignment:**
- Load peak ground acceleration observations from the 2014 Napa Earthquake
- Fit a ground motion prediction equation (GMPE) using `polyfit`
- Fit a GMPE after weighing the data by the distance distribution using `linalg.solve`
- Compare those models which are based on data from one earthquake with the GMPE of Abrahamson and Silva (2008) which was fit using data from 140 earthquakes of various magnitudes.


## Setup

Run this cell as it is to setup your environment.

In [ ]:
import math
import numpy as np
import pandas as pd
from scipy import stats
import matplotlib.pyplot as plt

## Exercise 1, GMPE Fitting

As we did in class, fit a GMPE model to the PGA data, for this assingment just use the data from the Napa event.

### Load and Plot Peak Ground Acceleration Data

Load the PGA data for just the 2014 Napa Earthquake.





In [ ]:
#Read Napa and Parkfield Earthquake Peak Ground Acceleration Data
dist, pga=np.array(pd.read_table('napa_pga.txt')).transpose()


Make a plot showing the data in both a linear-linear, and loglog projection. Note that the data is also non-linear even in the log-log projection. 

In [ ]:
#Plot the two data sets


### Fitting Strong Motion Data

In order to use the observations of peak ground acceleration (and other parameters like peak velocity, or spectral acceleration quantities) it is necessary to develop a model that accurately describes the behavior. From physics it is understood that in the far-field (large distance compared to the source dimension) that ground motions decay as a power law with distance due to the spreading of wave energy in three dimensions as the wavefield travels outward from the earthquake source. This is called geometrical spreading. In addition, there is a inelastic attenuation term that accounts for dissipative energy loss due to material imperfections. Based on theory the following is a simple relationship that describes this behavior.

$pga=a*{\frac{1}{r^b}}*e^{cr}$

$r=\sqrt{(dist^2 + h^2)}$

$r$ is the total distance from the source taking into account an average depth $h$, $a$ is a coeffient that depends on magnitude and scales the overall motions, $b$ is the exponent for the power-law geometrical spreading term, and $c$ is the coefficient for the in-elastic term (important only at large distances). Taking the natural logarithm of this equation yields a linear relationship in the model coeffients.

$\mathrm{ln}(pga)=a + b*\mathrm{ln}(r) + c*r$

For this exercise we will fit the above equation assuming that $c=0$ to the data. 

__Compute the ground motion prediction equation (GMPE) for only the 2014 Napa data set. Use a $h =4.0$.__



In [ ]:
#Write code for fitting the GMPE to the data here


Compute the 95% confidence intervals for the unweighted model.

In [ ]:
#Compute 95% confidence levels


Plot the results and print the best fitting solution parameters.

In [ ]:
#Plot Inversion Results


How does this model compare to what we found using both datasets in class? 

_Write your answer here._

### Weighted Fitting

You have probably noticed that the data is not uniformly distributed in distance. In such circumstances it is often advantageous to weight the data prior to inversion to make their respecive influence to the model more equal. In this particular problem it is possible to consider discrete distance bins of 10 km width from 0 to 180 km and count the number of events in each bin. Then the weight can be the inverse of the count, meaning that the weight of bins with many observations will be less and distance bins with fewer will be given more influence with a goal of making the influence on the model more equal over the entire distance range.

__Compute the ground motion prediction equation (GMPE) for only the 2014 Napa data set using the number of observations per distance bin to weigh the data. Use a $h =4.0$.__




In [ ]:
#Determine distance weighting bins


In [ ]:
#Write code for fitting the GMPE to the data here

Compute 95% confidence levels for the weighted model. 

In [ ]:
#Compute 95% confidence levels


Plot the Napa data and both the unweighted and weighted models. Print the best fitting solution parameters.  

In [ ]:
#Plot Inversion Results


Compare your results with and without the weighting scheme.

_Write your answer here._

## Exercise 2, Abrahamson and Silva (2008) GMPE

The GMPE that you developed for the Napa earthquake data set is actually quite good, but it is limited to only one M6 earthquake. Abrahamson and Silva (2008, AS2008) developed a GMPE considering 2750 recordings from 140 earthquakes ranging in magnitude from 4.27 to 7.62. They report that the derived GMPE is applicable to M 5.0 to 8.5 earthquakes. The following shows the AS2008 relationship for a M7.5 earthquake.

<img src="./as2008.png">

In the following cell the definition AS2008 GMPE for hard rock, considering, distance, magnitude, and the depth to the top of the fault is given. __The function takes three input arguments, an array of distances, a magnitude and the depth to the top of the fault. The output is the natural logarithm of median peak ground acceleration from the AS2008 GMPE (lnpga), and the 95% confidence level (sigma).__ For M6.5+ events in California we can consider the top of the fault to be at zero depth. A M5 may be at 8 km in comparison.


In [ ]:
def as2008(dist,M,Ztor):
    """
    This function takes an array of distances (only horizontal distance, h=0), a magnitude and the depth
    and returns the natural logarithm of median peak ground acceleration from the AS2008 GMPE, and the
    standard deviation (sigma).
    
    The function is not the complete AS2008 formulation. It is limited to the hard rock case Vs30=865, and only
    computes pga.
    
    """
    #Defined by A&S2008 DO NOT CHANGE
    c1=6.75;
    c4=4.5;
    a1=0.804;   #for PGA only this parameter is period dependent
    a2=-0.9679; #for PGA only this parameter is period dependent
    a3=0.265;
    a4=-0.231;
    a5=-0.398;
    a8=-0.0372; #for PGA only this parameter is period dependent
    a16=0.9000; #for PGA only
    VLIN=865.1; #for PGA only note for vs30=vlin f5==0
    #Defined by A&S2008 DO NOT CHANGE
    
    R=np.sqrt(dist*dist + c4*c4)      #compute total distance

    #Standard Deviation varies from 0.8 for M5 to 0.6 for M7 assume linear in ln
    if M <= 7 and M >= 5:
        sigma=0.8+(0.8-0.6)/(5-7)*(M-5)

    if M > 7:
        sigma=0.6;

    #Base model
    if M <= c1:
        f1=a1+a4*(M-c1)+a8*(8.5-M)**2+(a2+a3*(M-c1))*np.log(R)

    if M > c1:
        f1=a1+a5*(M-c1)+a8*(8.5-M)**2+(a2+a3*(M-c1))*np.log(R)

    #Depth of fault
    if Ztor <= 10:
        f6=Ztor/10*a16

    if Ztor > 10:
        f6=a16


    lnpga=f1 + f6
    
    return lnpga, sigma

Use the `as2008` function defined above to compute the GMPE for a M6 earthquake with a top fault depth of 1.0km.

Plot this AS2008 model for a M6 earthquake at 1km depth along with the data and your unwighted and weighted least squares models.

In [ ]:
#Plot Results


How does AS2008 fit the data, and how does it compare to the GMPE model you developed by inverting the data? Are your models within the 95% confidence bounds of AS2008?

_Write your answer here._

Explore the effect of earthquake magnitude on ground acceleration. Use the `as2008` function defined above to compute the GMPE for M5, M6, M7, and M8 earthquakes (all with a top fault depth of 1.0km). Plot them together.

How does the ground acceleration curve change with increasing earthquake magnitude?

_Write your answer here._

Explore the effect of fault depth on ground acceleration. Use the `as2008` function defined above to compute the GMPE for M6 earthquakes with a top fault depth of 0.0, 1.0, and 4.0 km. Plot them together.

How does the ground acceleration curve change with increasing fault depth?

_Write your answer here._